In [3]:
#final

import pandas as pd
from datetime import datetime


def clean_currency(value):
    if pd.notna(value) and isinstance(value, str):
        
        return value.replace('$', '').replace(',', '').strip()
    return value


contacts_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_contacts_data.xlsx')
owner_contacts_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\owner_contacts.xlsx')
properties_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_properties_data.xlsx')


owner_folios_data = []

# Start id from 1001
current_id = 1001


for index, row in contacts_df.iterrows():
    
    folio_code_column = row['Inner_Owner Folio code 2']
    if pd.notna(folio_code_column):
    
        folio_code = folio_code_column.split('(')[1].split(')')[0] if '(' in folio_code_column else None

       
        owner_contact_ref = folio_code_column.split('(')[0].strip()
        owner_contact_id = owner_contacts_df.loc[owner_contacts_df['reference'] == owner_contact_ref, 'id'].values[0] if owner_contact_ref in owner_contacts_df['reference'].values else None

        
        property_ref = row['Inner_Owner Property 2']
        property_id = properties_df.loc[properties_df['Reference'] == property_ref, 'properties_row'].values[0] if property_ref in properties_df['Reference'].values else None

        
        next_disburse_date = row['Inner_Owner Next disbursement 2']
        if pd.notna(next_disburse_date):
            try:
                next_disburse_date = pd.to_datetime(next_disburse_date, format='%d/%m/%Y').strftime('%Y-%m-%d')
            except ValueError:
                next_disburse_date = None

        
        owner_folios_data.append([
            current_id,  # Auto-incremental id starting from 1001
            0,  # total_money
            0,  # balance
            row['Inner_Owner Disbursements 2'],  # regular_intervals
            next_disburse_date,  # next_disburse_date
            clean_currency(row['Inner_Owner Withhold amount 2']),  # withhold_amount (without $)
            'NULL',  # withold_reason
            'NULL',  # agreement_start
            'NULL',  # gained_reason
            'NULL',  # comment
            'NULL',  # agreement_end
            'NULL',  # owner_access
            owner_contact_id,  # owner_contact_id
            datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # created_at
            datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # updated_at
            clean_currency(row['Folio_Paid To']) if pd.notna(folio_code_column) else None,  # money_in (without $ and ,)
            clean_currency(row['Folio_Part Paid']) if pd.notna(folio_code_column) else None,  # money_out (without $ and ,)
            clean_currency(row['Folio_Bond Held']) if pd.notna(folio_code_column) else None,  # uncleared (without $ and ,)
            folio_code,  # folio_code
            property_id,  # property_id
            1,  # company_id
            clean_currency(row['Folio_MePay Pending']) if pd.notna(folio_code_column) else None,  # opening_balance (without $ and ,)
            1,  # status
            'NULL',  # fee_status
            clean_currency(row['Folio_Owner Balance']) if pd.notna(folio_code_column) else None  # total_balance (without $ and ,)
        ])

        # Increment the id for the next row
        current_id += 1


owner_folios_df = pd.DataFrame(owner_folios_data, columns=[
    'Id', 'total_money', 'balance', 'regular_intervals', 'next_disburse_date', 'withhold_amount', 
    'withold_reason', 'agreement_start', 'gained_reason', 'comment', 'agreement_end', 
    'owner_access', 'owner_contact_id', 'created_at', 'updated_at', 'money_in', 'money_out', 
    'uncleared', 'folio_code', 'property_id', 'company_id', 'opening_balance', 'status', 
    'fee_status', 'total_balance'
])


owner_folios_df.to_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\owner_folios.xlsx', index=False)

print("owner_folios.xlsx file created successfully with the specified modifications.")


owner_folios.xlsx file created successfully with the specified modifications.
